### Generating Guides from OpenAPI Specification

*iteratively create more complicated examples to prototype generation of high quality guides*

### 1️⃣ Rudimentary Example

```mermaid
graph LR
    allow.com.yaml --> DocumentLoader
    DocumentLoader --> Chat
    Query[How do I create a journey in Python?] --> Chat
    Chat --> Guide
```

In [16]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain


chat = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0.2)
question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "When answering questions, use markdown to ensure code blocks and commands are propertly formatted. Answer the user's questions based on the below context:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
document_chain = create_stuff_documents_chain(chat, question_answering_prompt)
from langchain.document_loaders.text import TextLoader

file_path = 'alloy.com.yaml'
loader = TextLoader(file_path)
docs = loader.load()
from langchain.memory import ChatMessageHistory

demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message("How do I create a journey in Python?")

output = document_chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
        "context": docs,
    }
)
from IPython.display import display, Markdown

display(Markdown(output))

To create a journey in Python, you would typically interact with an API that supports journey creation. Based on the provided API documentation, it appears you want to create a journey application using the Alloy API. Here's how you can do it using Python and the `requests` library. 

First, ensure you have the `requests` library installed. If not, you can install it using pip:

```bash
pip install requests
```

Next, you can use the following Python script to create a journey application. This example assumes you have a journey token (`journey_token`) and you're creating a journey application for a person entity. You'll need to replace `YOUR_JOURNEY_TOKEN`, `YOUR_APPLICATION_TOKEN`, and `YOUR_APPLICATION_SECRET` with your actual journey token, application token, and application secret, respectively.

```python
import requests
from requests.auth import HTTPBasicAuth
import json

# Replace these variables with your actual details
journey_token = 'YOUR_JOURNEY_TOKEN'
application_token = 'YOUR_APPLICATION_TOKEN'
application_secret = 'YOUR_APPLICATION_SECRET'
api_url = f'https://demo-qasandbox.alloy.co/v1/journeys/{journey_token}/applications'

# Prepare the headers
headers = {
    'Content-Type': 'application/json',
    # Additional headers can be added here if needed
}

# Prepare the data for the person entity
data = {
    "entities": [
        {
            "external_entity_id": "person-entity",
            "gender": "male",
            "birth_date": "1998-02-03",
            "name_first": "John",
            "name_last": "Doe",
            "document_ssn": "123456789",
            "addresses": [
                {
                    "address_line_1": "41 E. 11th",
                    "address_city": "New York",
                    "address_state": "NY",
                    "address_postal_code": "10003",
                    "address_country_code": "US",
                    "type": "primary"
                }
            ],
            "emails": [
                {
                    "email_address": "john@alloy.com"
                }
            ],
            "phones": [
                {
                    "phone_number": "555-000-1234"
                }
            ],
            "pii_status": "successful"
        }
    ]
}

# Make the request
response = requests.post(api_url, headers=headers, auth=HTTPBasicAuth(application_token, application_secret), data=json.dumps(data))

# Check the response
if response.status_code == 201:
    print("Journey application created successfully.")
    print(response.json())
else:
    print(f"Failed to create journey application. Status code: {response.status_code}")
    print(response.text)
```

This script sends a POST request to the Alloy API to create a journey application for a person entity. It includes basic information such as name, birth date, SSN, address, email, and phone number. 

Make sure to handle the response appropriately based on your application's needs, and always secure your application token and secret properly.

### gpt-4-turbo Pricing

- Input: $10.00 / 1M tokens

- Output: $30.00 / 1M tokens

### gpt-3.5-turbo Pricing

- Input: $0.50 / 1M tokens

- Output: $1.50 / 1M tokens

In [17]:
import tiktoken

# To get the tokeniser corresponding to a specific model in the OpenAI API:
enc = tiktoken.encoding_for_model("gpt-4")

with open("alloy.com.yaml") as f:
    doc = f.read()

tokens = enc.encode(doc)
len(tokens)
gpt4_price = len(tokens) * 10 / 1_000_000
gpt35_price = len(tokens) * 0.5 / 1_000_000
print("gpt-4-turbo: Price of passing in alloy.com.yaml 1 time: ${}".format(gpt4_price))
print("gpt-4-turbo: Price of passing in alloy.com.yaml 20 times: ${}".format(gpt4_price * 20))
print("gpt-3.5-turbo: Price of passing in alloy.com.yaml 1 time: ${}".format(gpt35_price))
print("gpt-3.5-turbo: Price of passing in alloy.com.yaml 20 times: ${}".format(gpt35_price * 20))

gpt-4-turbo: Price of passing in alloy.com.yaml 1 time: $0.50122
gpt-4-turbo: Price of passing in alloy.com.yaml 20 times: $10.0244
gpt-3.5-turbo: Price of passing in alloy.com.yaml 1 time: $0.025061
gpt-3.5-turbo: Price of passing in alloy.com.yaml 20 times: $0.50122


### ⛔️ Problem #1: Passing entire OpenAPI Specification is too expensive
### ⛔️ Problem #2: Entire OpenAPI Specification does not fit in gpt-3.5-turbo
### 🤔 Solution: Chunk the OpenAPI Specification, retrieve from vector store, and pass in as context instead

#### Data Pipeline

```mermaid
graph LR
    alloy.com.yaml --> Chunk1
    alloy.com.yaml --> Chunk2
    alloy.com.yaml --> Chunk3
    Chunk1 -->|embed| VectorStore
    Chunk2 -->|embed| VectorStore
    Chunk3 -->|embed| VectorStore
    VectorStore -->|How do I create a journey in Python?| Chunk1Out[Chunk1]
    VectorStore -->|How do I create a journey in Python?| Chunk3Out[Chunk3]
    Chunk1Out --> Context
    Chunk3Out --> Context
    Context --> Chat
    Query[How do I create a journey in Python?] --> Chat
    Chat --> Guide
```

#### RAG Chain

```mermaid
graph LR
    alloy.com.yaml --> TextSplitter
    TextSplitter --> Chroma[Chroma 'Vector Store']
    Chroma --> Retriever
    Retriever --> Context
    Context --> Chat
    Query[How do I create a journey in Python?] --> Chat
    Chat --> Guide
```

In [90]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, model_kwargs={"seed": 0})

In [98]:
from langchain.document_loaders.text import TextLoader
def init_docs():
    file_path = 'alloy.com.yaml'
    loader = TextLoader(file_path)
    docs = loader.load()
    return docs

In [99]:
docs = init_docs()

In [100]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# store OAS in vector DB
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

In [101]:
query = "How do I create a journey in Python?"

In [102]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [103]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | chat
    | StrOutputParser()
)

output = rag_chain.invoke(query)

In [104]:
from IPython.display import display, Markdown

display(Markdown(output))

To create a journey in Python, you can use the provided API endpoints and parameters to define the journey name, type, token, and version number. You can then make a POST request to the specified endpoint with the necessary data to create the journey application note. The API documentation provides details on the required parameters and response structure for creating a journey application note.

### 👎 Results 

Output is nearly useless. Does not provide environment setup instructions or example code.

### 🤔 Maybe provide system instructions that are more explicit?

In [105]:
from langchain_core.prompts import ChatPromptTemplate

better_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an assistant for question-answering tasks. f you don't know the answer, just say that you don't know. Use markdown to ensure code blocks and commands are propertly formatted. Make sure to always include environment setup instructions and code blocks that are helpful for a developer to copy-paste. Always explain inputs and outputs of API requests. Be as detailed as possible. Answer the user's questions based on the below context:\n\n{context}",
        ),
        (
            "user",
            "{question}"
        )
    ]
)

In [106]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | better_prompt
    | chat
    | StrOutputParser()
)

In [107]:
output = rag_chain.invoke(query)

In [108]:
display(Markdown(output))

To create a journey in Python using the Alloy API, you can follow these steps:

1. **Set up Authorization**:
   - You need to have a valid workflow token and secret to authenticate your requests. The Alloy API supports OAuth2 for authentication.
   - You can use the OAuth2 flow to generate a bearer token using your workflow token and secret.
   - Make sure you have the necessary permissions to create a journey.

2. **Make a POST Request to Create a Journey**:
   - Use the endpoint `/journeys` to create a new journey.
   - Provide the required parameters in the request body, such as `journey_name`, `journey_type`, and any other relevant information.
   - Ensure that the request is properly authenticated with the bearer token.

Here is an example code snippet in Python using the `requests` library to create a journey:

```python
import requests

url = "https://demo-qasandbox.alloy.co/v1/journeys"

headers = {
    "Authorization": "Bearer YOUR_BEARER_TOKEN",
    "Content-Type": "application/json"
}

data = {
    "journey_name": "My New Journey",
    "journey_type": "application",
    "journey_version_num": "1.0"
}

response = requests.post(url, headers=headers, json=data)

if response.status_code == 201:
    print("Journey created successfully!")
    journey_data = response.json()
    print("Journey ID:", journey_data.get("journey_token"))
else:
    print("Failed to create journey. Status code:", response.status_code)
    print("Error message:", response.text)
```

Make sure to replace `YOUR_BEARER_TOKEN` with the actual bearer token generated using your workflow token and secret.

This code snippet sends a POST request to create a new journey with the specified name and type. If the request is successful, it will print the journey ID. Otherwise, it will print the error message.

Remember to handle any exceptions and error cases based on the API response for a robust implementation.

### 👎 Results

Looks like the result is lacking the correct base URL, proper parameters, and explanation of inputs and outputs. This makes the answer still nearly useless. Likely because the model was not given enough context.

### 🤔 Lets investigate what our vector store is returning

In [109]:
results = retriever.invoke(query)

for doc in results:
    print(doc.page_content)

type: object
                                  properties:
                                    href:
                                      type: string
                      journey:
                        type: object
                        properties:
                          journey_name:
                            type: string
                          journey_type:
                            type: string
                            enum:
                              - application
                              - alert
                          journey_token:
                            type: string
                          journey_version_num:
                            type: string
                          _links:
                            type: object
                            properties:
                              self:
                                type: object
                                properties:
properties:
                          self:
            

### 👎 Useless chunk results

The results don't contain anything about the relevant API endpoint

### 🤔 Can we improve the chunk results by doing smarter chunking?

Maybe if we chunk the OpenAPI spec based on operations, we can give the LLM better context. For smarter chunking, we can try to chunk the OpenAPI into documents that only contain relevant information for a specific operation but keep all the contextual information (everything besides `paths`)

In [110]:
import yaml
import jsonref
from jsonref import replace_refs
from langchain_core.documents.base import Document
from copy import deepcopy
from pprint import pprint

with open("alloy.com.yaml") as f:
    spec = f.read()

def chunk_openapi_by_operation(openapi: str):
    parsed = yaml.safe_load(openapi)

    operations: (str, str) = []
    # 1) list all operations by (path, HTTP method)
    for path, methods in parsed['paths'].items():
        for method in methods.keys():
            # if method is not an HTTP method then skip
            if method.lower() not in ['get', 'post', 'put', 'delete', 'patch', 'head', 'options', 'trace']:
                continue
            operations.append((path, method))

    # 2) create a chunk for every operation

    # 2.a) Dereference entire OpenAPI Spec
    dereferenced = replace_refs(parsed, lazy_load=False)

    chunks = []
    for operation in operations:
        path = operation[0]
        method = operation[1]
        chunk = deepcopy(dereferenced)
        if 'tags' in chunk['paths'][operation[0]][operation[1]]:
            tags = chunk['paths'][operation[0]][operation[1]]['tags']

        # first tag if possible
        if tags:
            tag_name = tags[0]

        # delete all tags on OAS except tag for this operation
        while len(chunk['tags']) > 1:
            for i in range(len(chunk['tags']) - 1, -1, -1):
                if chunk['tags'][i]['name'] != tag_name:
                    chunk['tags'].pop(i)

        if "summary" in chunk['paths'][path][method]:
            summary = chunk['paths'][path][method]['summary']
        else:
            summary = ""

        if "description" in chunk['paths'][path][method]:
            description = chunk['paths'][path][method]['description']
        else:
            description = ""

        # delete other operations
        for other_operation in operations:
            if other_operation[0] == operation[0]:
                continue
            if other_operation[0] in chunk['paths']:
                del chunk['paths'][other_operation[0]]

        # delete empty paths
        for path in chunk['paths'].keys():
            if not chunk['paths'][path]:
                del chunk['paths'][path]

        # delete other operations under same path
        keys = list(chunk['paths'][operation[0]].keys())
        for method in keys:
            if operation[1] == method:
                continue
            del chunk['paths'][operation[0]][method]

        # delete all components (should be inlined from 2.a)
        del chunk['components']
        chunks.append(({
            "path": operation[0],
            "method": operation[1],
            "openapi": yaml.dump(chunk),
            "tag": tag_name,
            "summary": summary,
            "description": description
        }))
    return list(map(lambda chunk: Document(page_content=chunk["openapi"], metadata={
        "path": chunk["path"],
        "method": chunk["method"],
        "tag": chunk["tag"],
        "summary": chunk["summary"],
        "description": chunk["description"]
    }), chunks))
chunks = chunk_openapi_by_operation(spec)
# for chunk in chunks:
#     print(len(yaml.safe_load(chunk.page_content)['paths']))
print(len(chunks))

72


In [111]:
# reset chroma DB
from langchain_community.vectorstores.chroma import Chroma

Chroma().delete_collection()

In [112]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

# for chunk in chunks:
#     print(len(yaml.safe_load(chunk.page_content)['paths']))

query = "How do I create a journey application in Python?"

chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, model_kwargs={"seed": 0})

# Reset the collection to remove embeddings from previous runs
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(chunks)
vectorstore = Chroma.from_documents(chunks, OpenAIEmbeddings())

In [113]:
metadata_field_info = [
    AttributeInfo(
        name="path",
        description="The subpath for this operation",
        type="string",
    ),
    AttributeInfo(
        name="method",
        description="The HTTP Method for this operation",
        type="string",
    ),
    AttributeInfo(
        name="tag",
        description="The logical grouping for this API operation",
        type="string",
    ),
    AttributeInfo(
        name="summary",
        description="A short description of this operation's functionality",
        type="string",
    ),
    AttributeInfo(
        name="description",
        description="A more detailed description of this operation's functionality",
        type="string",
    ),
]
document_content_description = "The pruned OpenAPI specification which includes only the relevant information for a particular operation in the OpenAPI specification."

# Dylan: really stupid, but I had to upgrade langchain to get this to work for some reason
retriever = SelfQueryRetriever.from_llm(
    chat, vectorstore, document_content_description, metadata_field_info, verbose=True, search_kwargs={"k": 4}
)

In [114]:
relevant_docs = retriever.invoke(query)
pprint(list(map(lambda doc: doc.metadata, relevant_docs)))

[{'description': 'Create a journey application for one or more entities.\n',
  'method': 'post',
  'path': '/journeys/{journey_token}/applications',
  'summary': 'Create Journey Application',
  'tag': 'Journeys'},
 {'description': 'Create a note associated with the specified Journey '
                 'Application',
  'method': 'post',
  'path': '/journeys/{journey_token}/applications/{journey_application_token}/notes',
  'summary': 'Create Journey Application Note',
  'tag': 'Journeys'},
 {'description': 'If a journey application has the status '
                 '`pending_journey_application_review`, this endpoint can be '
                 'used to inform the system of the outcome of the manual '
                 'review and submit review notes. The outcome submitted here '
                 'will be the final outcome of the journey application.',
  'method': 'post',
  'path': '/journeys/{journey_token}/applications/{journey_application_token}/review',
  'summary': 'Manual Review Jour

In [115]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

better_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-class solutions engineer and amazing at question-answering tasks. If you don't know the answer, just say that you don't know. Use markdown to ensure code blocks and commands are propertly formatted. Make sure to always include environment setup instructions and code blocks that are helpful for a developer to copy-paste. Always explain inputs and outputs of API requests. Be as detailed as possible. Answer the user's questions based on the below context:\n\n{context}",
        ),
        (
            "user",
            "{question}"
        )
    ]
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | better_prompt
    | chat
    | StrOutputParser()
)

In [116]:
from IPython.display import display, Markdown
output = rag_chain.invoke(query)
display(Markdown(output))

To create a journey application using the Alloy API in Python, you will need to send a POST request to the `/journeys/{journey_token}/applications` endpoint with the required payload. Here's a step-by-step guide on how to do this:

### Prerequisites:
- You need to have the `requests` library installed. If you don't have it installed, you can install it using pip:
  ```bash
  pip install requests
  ```

### Python code to create a journey application:
```python
import requests
import json

# Define the endpoint URL and journey token
url = "https://demo-qasandbox.alloy.co/v1/journeys/{journey_token}/applications"
journey_token = "your_journey_token_here"

# Define the payload for creating a journey application
payload = {
    "do_await_additional_entities": False,
    "entities": [
        {
            "branch_name": "persons",
            "data": {
                "addresses": [
                    {
                        "city": "New York",
                        "country_code": "US",
                        "line_1": "41 E. 11th",
                        "line_2": "2nd floor",
                        "postal_code": "10003",
                        "state": "NY",
                        "type": "primary"
                    }
                ],
                "birth_date": "1990-01-25",
                "document_ssn": "111223333",
                "email_address": "john@alloy.com",
                "ip_address_v4": "42.206.213.70",
                "meta": {
                    "user_type": "vip"
                },
                "name_first": "John",
                "name_last": "Doe",
                "name_middle": "Franklin",
                "phone_number": "8443825569"
            },
            "entity_type": "person",
            "external_entity_id": "my_system_entity_id_123"
        }
    ]
}

# Convert the payload to JSON
payload_json = json.dumps(payload)

# Send the POST request to create a journey application
response = requests.post(url.format(journey_token=journey_token), json=payload_json)

# Print the response
print(response.json())
```

In the code snippet above:
- Replace `"your_journey_token_here"` with the actual journey token you want to create the application for.
- Update the `payload` dictionary with the necessary data for creating the journey application.

After running this Python script, you should be able to create a journey application using the Alloy API.

### 👎 Looks like we are missing security requirements

Notice how the naive approach includes directions on how to authenticate with the endpoint as well. One thing that the naive approach doesn't include is both authentication options: basic & OAuth.

### 🤔 Lets make sure the model has all the necessary information to infer security requirements

In [117]:
relevant_docs = retriever.invoke(query)
print(relevant_docs[0].page_content)

_id: 626c513d1c9b07002812708b
info:
  description: hey hey hey, it's the Alloy API!
  title: Alloy API
  version: 1.0.0
openapi: 3.0.0
paths:
  /journeys/{journey_token}/applications:
    post:
      description: 'Create a journey application for one or more entities.

        '
      requestBody:
        content:
          application/json:
            schema:
              properties:
                do_await_additional_entities:
                  default: false
                  description: 'If this value is true, additional entities can be
                    sent after this request by using the PUT endpoint that updates
                    a journey application.


                    The journey application will not complete until the parameter
                    `has_finished_sending_additional_entities` is sent with the value
                    `true` to the PUT endpoint.

                    '
                  example: false
                  type: boolean
                e

### 💡 Since we are deleting the components from OpenAPI spec, it loses context about security requirements

### 👉 Preserve components.securitySchemes

In [118]:
import yaml
import jsonref
from jsonref import replace_refs
from langchain_core.documents.base import Document
from copy import deepcopy
from pprint import pprint

with open("alloy.com.yaml") as f:
    spec = f.read()

def chunk_openapi_by_operation(openapi: str):
    parsed = yaml.safe_load(openapi)

    operations: (str, str) = []
    # 1) list all operations by (path, HTTP method)
    for path, methods in parsed['paths'].items():
        for method in methods.keys():
            # if method is not an HTTP method then skip
            if method.lower() not in ['get', 'post', 'put', 'delete', 'patch', 'head', 'options', 'trace']:
                continue
            operations.append((path, method))

    # 2) create a chunk for every operation

    # 2.a) Dereference entire OpenAPI Spec
    dereferenced = replace_refs(parsed, lazy_load=False)

    chunks = []
    for operation in operations:
        path = operation[0]
        method = operation[1]
        chunk = deepcopy(dereferenced)
        if 'tags' in chunk['paths'][operation[0]][operation[1]]:
            tags = chunk['paths'][operation[0]][operation[1]]['tags']

        # first tag if possible
        if tags:
            tag_name = tags[0]

        # delete all tags on OAS except tag for this operation
        while len(chunk['tags']) > 1:
            for i in range(len(chunk['tags']) - 1, -1, -1):
                if chunk['tags'][i]['name'] != tag_name:
                    chunk['tags'].pop(i)

        if "summary" in chunk['paths'][path][method]:
            summary = chunk['paths'][path][method]['summary']
        else:
            summary = ""

        if "description" in chunk['paths'][path][method]:
            description = chunk['paths'][path][method]['description']
        else:
            description = ""

        # delete other operations
        for other_operation in operations:
            if other_operation[0] == operation[0]:
                continue
            if other_operation[0] in chunk['paths']:
                del chunk['paths'][other_operation[0]]

        # delete empty paths
        for path in chunk['paths'].keys():
            if not chunk['paths'][path]:
                del chunk['paths'][path]

        # delete other operations under same path
        keys = list(chunk['paths'][operation[0]].keys())
        for method in keys:
            if operation[1] == method:
                continue
            del chunk['paths'][operation[0]][method]

        # delete all components besides securitySchemes (should be inlined from 2.a)
        if "components" in chunk:
            for key in chunk["components"]:
                if key == "securitySchemes":
                    continue
                del chunk['components'][key]
        
        chunks.append(({
            "path": operation[0],
            "method": operation[1],
            "openapi": yaml.dump(chunk),
            "tag": tag_name,
            "summary": summary,
            "description": description
        }))
    return list(map(lambda chunk: Document(page_content=chunk["openapi"], metadata={
        "path": chunk["path"],
        "method": chunk["method"],
        "tag": chunk["tag"],
        "summary": chunk["summary"],
        "description": chunk["description"]
    }), chunks))
chunks = chunk_openapi_by_operation(spec)
# for chunk in chunks:
#     print(len(yaml.safe_load(chunk.page_content)['paths']))
print(len(chunks))

72


In [119]:
# reset chroma DB
from langchain_community.vectorstores.chroma import Chroma

Chroma().delete_collection()

In [120]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

# for chunk in chunks:
#     print(len(yaml.safe_load(chunk.page_content)['paths']))

query = "How do I create a journey application in Python?"

chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, model_kwargs={"seed": 0})

# Reset the collection to remove embeddings from previous runs
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(chunks)
vectorstore = Chroma.from_documents(chunks, OpenAIEmbeddings())

In [121]:
metadata_field_info = [
    AttributeInfo(
        name="path",
        description="The subpath for this operation",
        type="string",
    ),
    AttributeInfo(
        name="method",
        description="The HTTP Method for this operation",
        type="string",
    ),
    AttributeInfo(
        name="tag",
        description="The logical grouping for this API operation",
        type="string",
    ),
    AttributeInfo(
        name="summary",
        description="A short description of this operation's functionality",
        type="string",
    ),
    AttributeInfo(
        name="description",
        description="A more detailed description of this operation's functionality",
        type="string",
    ),
]
document_content_description = "The pruned OpenAPI specification which includes only the relevant information for a particular operation in the OpenAPI specification."

# Dylan: really stupid, but I had to upgrade langchain to get this to work for some reason
retriever = SelfQueryRetriever.from_llm(
    chat, vectorstore, document_content_description, metadata_field_info, verbose=True, search_kwargs={"k": 4}
)

In [122]:
relevant_docs = retriever.invoke(query)
print(relevant_docs[0].page_content)

_id: 626c513d1c9b07002812708b
components:
  securitySchemes:
    basic:
      description: HTTP basic authorization using a workflow token and secret
      scheme: Basic
      type: http
    oauth2:
      description: Oauth2 using a workflow token and secret to generate a bearer token
      flows:
        clientCredentials:
          tokenUrl: /oauth/bearer
      type: oauth2
      x-default: viSRrSuUJEid8u0l3dyRTj5ATsWpHX9ShD51TH3j
info:
  description: hey hey hey, it's the Alloy API!
  title: Alloy API
  version: 1.0.0
openapi: 3.0.0
paths:
  /journeys/{journey_token}/applications:
    post:
      description: 'Create a journey application for one or more entities.

        '
      requestBody:
        content:
          application/json:
            schema:
              properties:
                do_await_additional_entities:
                  default: false
                  description: 'If this value is true, additional entities can be
                    sent after this reques

In [123]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

better_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-class solutions engineer and amazing at question-answering tasks. If you don't know the answer, just say that you don't know. Use markdown to ensure code blocks and commands are propertly formatted. Make sure to always include environment setup instructions and code blocks that are helpful for a developer to copy-paste. Always explain inputs and outputs of API requests. Be as detailed as possible. Answer the user's questions based on the below context:\n\n{context}",
        ),
        (
            "user",
            "{question}"
        )
    ]
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | better_prompt
    | chat
    | StrOutputParser()
)

In [124]:
from IPython.display import display, Markdown
output = rag_chain.invoke(query)
display(Markdown(output))

To create a journey application in Python, you will need to make an HTTP POST request to the specified endpoint `/journeys/{journey_token}/applications` with the necessary payload containing the entities and other required information. Here's a step-by-step guide on how to achieve this using the `requests` library in Python:

### Prerequisites:
Make sure you have the `requests` library installed. If not, you can install it using pip:
```bash
pip install requests
```

### Python code to create a journey application:
```python
import requests
import json

# Define the endpoint URL and journey token
url = "https://demo-qasandbox.alloy.co/v1/journeys/{journey_token}/applications"
journey_token = "your_journey_token_here"

# Define the payload for creating a journey application
payload = {
    "do_await_additional_entities": False,
    "entities": [
        {
            "branch_name": "persons",
            "data": {
                "addresses": [
                    {
                        "city": "New York",
                        "country_code": "US",
                        "line_1": "41 E. 11th",
                        "line_2": "2nd floor",
                        "postal_code": "10003",
                        "state": "NY",
                        "type": "primary"
                    }
                ],
                "birth_date": "1990-01-25",
                "document_ssn": "111223333",
                "email_address": "john@alloy.com",
                "ip_address_v4": "42.206.213.70",
                "meta": {
                    "user_type": "vip"
                },
                "name_first": "John",
                "name_last": "Doe",
                "name_middle": "Franklin",
                "phone_number": "8443825569"
            },
            "entity_type": "person",
            "external_entity_id": "my_system_entity_id_123"
        }
    ]
}

# Convert the payload to JSON
payload_json = json.dumps(payload)

# Make the POST request to create the journey application
response = requests.post(url.format(journey_token=journey_token), data=payload_json, headers={"Content-Type": "application/json"})

# Check if the request was successful
if response.status_code == 201:
    print("Journey application created successfully!")
    print("Journey Application Token:", response.json().get("journey_application_token"))
else:
    print("Failed to create journey application. Status code:", response.status_code)
    print("Response:", response.text)
```

In the above Python code:
- Replace `{journey_token}` with your actual journey token.
- Update the `payload` dictionary with the entities and data you want to include in the journey application.
- The code sends a POST request to the Alloy API endpoint to create the journey application.
- It then checks the response status code and prints the journey application token if the request was successful.

Make sure to replace placeholders with actual values before running the code.

### 👎 The guide did not explicitly guide the user through authentication

### 🤔 Maybe better prompting will do the trick

In [125]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

better_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a world-class solutions engineer and amazing at question-answering tasks. You are also a world-class API
            integrations specialist and deeply understand OpenAPI Specification.
            
            You strictly follow these rules:
            - If you don't know the answer, just say that you don't know.
            - Use markdown to ensure code blocks and commands are propertly formatted.
            - Make sure to always include environment setup instructions and code blocks that are helpful for a developer to copy-paste.
            - Always explain inputs and outputs of API requests. Be as detailed as possible.
            - When presented with an OpenAPI Specification and asked questions about it, always include necessary steps to
            authenticate with the endpoint.
            - When presented with multiple authentication options, make sure to guide the customer through both methods
            
            Answer the user's questions based on the below context:\n\n{context}""",
        ),
        (
            "user",
            "{question}"
        )
    ]
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | better_prompt
    | chat
    | StrOutputParser()
)

In [126]:
from IPython.display import display, Markdown
output = rag_chain.invoke(query)
display(Markdown(output))

To create a journey application in Python using the Alloy API, you will need to make a POST request to the `/journeys/{journey_token}/applications` endpoint with the necessary payload containing the entities to be processed in the application.

Here's a step-by-step guide on how to create a journey application in Python:

1. Install the `requests` library if you haven't already. You can install it using pip:

```bash
pip install requests
```

2. Import the necessary libraries and define the endpoint URL, journey token, and payload for creating the journey application:

```python
import requests

url = "https://demo-qasandbox.alloy.co/v1/journeys/{journey_token}/applications"
journey_token = "your_journey_token_here"

payload = {
    "do_await_additional_entities": False,
    "entities": [
        {
            "branch_name": "persons",
            "data": {
                "addresses": [
                    {
                        "city": "New York",
                        "country_code": "US",
                        "line_1": "41 E. 11th",
                        "line_2": "2nd floor",
                        "postal_code": "10003",
                        "state": "NY",
                        "type": "primary"
                    }
                ],
                "birth_date": "1990-01-25",
                "document_ssn": "111223333",
                "email_address": "john@alloy.com",
                "ip_address_v4": "42.206.213.70",
                "meta": {
                    "user_type": "vip"
                },
                "name_first": "John",
                "name_last": "Doe",
                "name_middle": "Franklin",
                "phone_number": "8443825569"
            },
            "entity_type": "person",
            "external_entity_id": "my_system_entity_id_123"
        }
    ]
}
```

3. Make a POST request to the Alloy API endpoint to create the journey application:

```python
response = requests.post(url.format(journey_token=journey_token), json=payload, auth=('your_workflow_token', 'your_workflow_secret'))

if response.status_code == 201:
    print("Journey application created successfully!")
    print(response.json())
else:
    print("Failed to create journey application. Status code:", response.status_code)
    print(response.text)
```

Make sure to replace `'your_workflow_token'` and `'your_workflow_secret'` with your actual workflow token and secret for authentication.

This code snippet demonstrates how to create a journey application in Python using the Alloy API.

### 👎 Better prompting was not helpful, what if we switch out the model for gpt-4 instead of gpt-3.5

### 🤔 What is the cost of tokens for gpt-4 with our new RAG architecture

In [127]:
import tiktoken

# To get the tokeniser corresponding to a specific model in the OpenAI API:
enc = tiktoken.get_encoding("cl100k_base")

relevant_docs = retriever.invoke(query)

tokens = []
for doc in relevant_docs:
    tokens += enc.encode(doc.page_content)
len(tokens)
gpt4_price = len(tokens) * 10 / 1_000_000
gpt35_price = len(tokens) * 0.5 / 1_000_000
print("gpt-4-turbo: Price of passing in alloy.com.yaml 1 time: ${}".format(gpt4_price))
print("gpt-4-turbo: Price of passing in alloy.com.yaml 20 times: ${}".format(gpt4_price * 20))
print("gpt-3.5-turbo: Price of passing in alloy.com.yaml 1 time: ${}".format(gpt35_price))
print("gpt-3.5-turbo: Price of passing in alloy.com.yaml 20 times: ${}".format(gpt35_price * 20))

gpt-4-turbo: Price of passing in alloy.com.yaml 1 time: $0.12483
gpt-4-turbo: Price of passing in alloy.com.yaml 20 times: $2.4966
gpt-3.5-turbo: Price of passing in alloy.com.yaml 1 time: $0.0062415
gpt-3.5-turbo: Price of passing in alloy.com.yaml 20 times: $0.12483


### RAG vs. Long Context

| Model   | RAG     | Long Context |
|---------|---------|--------------|
| GPT-4   | \$0.124  | \$0.5 (4x)    |
| GPT-3.5 | \$0.006  | \$0.025 (4x)   |



### 👎 Still kind of expensive

### 🤔 What if we limit context to 2 documents

In [128]:
metadata_field_info = [
    AttributeInfo(
        name="path",
        description="The subpath for this operation",
        type="string",
    ),
    AttributeInfo(
        name="method",
        description="The HTTP Method for this operation",
        type="string",
    ),
    AttributeInfo(
        name="tag",
        description="The logical grouping for this API operation",
        type="string",
    ),
    AttributeInfo(
        name="summary",
        description="A short description of this operation's functionality",
        type="string",
    ),
    AttributeInfo(
        name="description",
        description="A more detailed description of this operation's functionality",
        type="string",
    ),
]
document_content_description = "The pruned OpenAPI specification which includes only the relevant information for a particular operation in the OpenAPI specification."

# Dylan: really stupid, but I had to upgrade langchain to get this to work for some reason
retriever = SelfQueryRetriever.from_llm(
    chat, vectorstore, document_content_description, metadata_field_info, verbose=True, search_kwargs={"k": 2}
)

In [129]:
import tiktoken

# To get the tokeniser corresponding to a specific model in the OpenAI API:
enc = tiktoken.get_encoding("cl100k_base")

relevant_docs = retriever.invoke(query)

tokens = []
for doc in relevant_docs:
    tokens += enc.encode(doc.page_content)
len(tokens)
gpt4_price = len(tokens) * 10 / 1_000_000
gpt35_price = len(tokens) * 0.5 / 1_000_000
print("gpt-4-turbo: Price of passing in alloy.com.yaml 1 time: ${}".format(gpt4_price))
print("gpt-4-turbo: Price of passing in alloy.com.yaml 20 times: ${}".format(gpt4_price * 20))
print("gpt-3.5-turbo: Price of passing in alloy.com.yaml 1 time: ${}".format(gpt35_price))
print("gpt-3.5-turbo: Price of passing in alloy.com.yaml 20 times: ${}".format(gpt35_price * 20))

gpt-4-turbo: Price of passing in alloy.com.yaml 1 time: $0.07569
gpt-4-turbo: Price of passing in alloy.com.yaml 20 times: $1.5137999999999998
gpt-3.5-turbo: Price of passing in alloy.com.yaml 1 time: $0.0037845
gpt-3.5-turbo: Price of passing in alloy.com.yaml 20 times: $0.07569000000000001


### RAG vs. Long Context

| Model   | RAG     | Long Context |
|---------|---------|--------------|
| GPT-4   | \$0.075  | \$0.5 (6.66x)    |
| GPT-3.5 | \$0.0038  | \$0.025 (6.66x)   |



### 🤔 Not bad, but does the output still include all the necessary info?

In [130]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

# for chunk in chunks:
#     print(len(yaml.safe_load(chunk.page_content)['paths']))

query = "How do I create a journey application in Python?"

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, model_kwargs={"seed": 0})

# Reset the collection to remove embeddings from previous runs
Chroma().delete_collection()

vectorstore = Chroma.from_documents(chunks, OpenAIEmbeddings())

In [131]:
metadata_field_info = [
    AttributeInfo(
        name="path",
        description="The subpath for this operation",
        type="string",
    ),
    AttributeInfo(
        name="method",
        description="The HTTP Method for this operation",
        type="string",
    ),
    AttributeInfo(
        name="tag",
        description="The logical grouping for this API operation",
        type="string",
    ),
    AttributeInfo(
        name="summary",
        description="A short description of this operation's functionality",
        type="string",
    ),
    AttributeInfo(
        name="description",
        description="A more detailed description of this operation's functionality",
        type="string",
    ),
]
document_content_description = "The pruned OpenAPI specification which includes only the relevant information for a particular operation in the OpenAPI specification."

# Dylan: really stupid, but I had to upgrade langchain to get this to work for some reason
retriever = SelfQueryRetriever.from_llm(
    chat, vectorstore, document_content_description, metadata_field_info, verbose=True, search_kwargs={"k": 2}
)

In [132]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

better_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a world-class solutions engineer and amazing at question-answering tasks. You are also a world-class API
            integrations specialist and deeply understand OpenAPI Specification.
            
            You strictly follow these rules:
            - If you don't know the answer, just say that you don't know.
            - Use markdown to ensure code blocks and commands are propertly formatted.
            - Make sure to always include environment setup instructions and code blocks that are helpful for a developer to copy-paste.
            - Always explain inputs and outputs of API requests. Be as detailed as possible.
            - When presented with an OpenAPI Specification and asked questions about it, always include necessary steps to
            authenticate with the endpoint.
            - When presented with multiple authentication options, make sure to guide the customer through both methods
            
            Answer the user's questions based on the below context:\n\n{context}""",
        ),
        (
            "user",
            "{question}"
        )
    ]
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | better_prompt
    | chat
    | StrOutputParser()
)

In [133]:
from IPython.display import display, Markdown
output = rag_chain.invoke(query)

display(Markdown(output))

To create a journey application in Python using the Alloy API, you will need to make a POST request to the `/journeys/{journey_token}/applications` endpoint with the necessary payload containing the entities to be processed in the application.

Here is a step-by-step guide to create a journey application in Python:

1. Install the `requests` library if you haven't already. You can install it using pip:

```bash
pip install requests
```

2. Import the necessary libraries and define the endpoint URL, journey token, and payload for creating the journey application:

```python
import requests

url = "https://demo-qasandbox.alloy.co/v1/journeys/{journey_token}/applications"
journey_token = "JourneyTokenHere"

payload = {
    "do_await_additional_entities": False,
    "entities": [
        {
            "branch_name": "persons",
            "data": {
                "addresses": [
                    {
                        "city": "New York",
                        "country_code": "US",
                        "line_1": "41 E. 11th",
                        "line_2": "2nd floor",
                        "postal_code": "10003",
                        "state": "NY",
                        "type": "primary"
                    }
                ],
                "birth_date": "1990-01-25",
                "document_ssn": "111223333",
                "email_address": "john@alloy.com",
                "ip_address_v4": "42.206.213.70",
                "meta": {
                    "user_type": "vip"
                },
                "name_first": "John",
                "name_last": "Doe",
                "name_middle": "Franklin",
                "phone_number": "8443825569"
            },
            "entity_type": "person",
            "external_entity_id": "my_system_entity_id_123"
        }
    ]
}
```

3. Make a POST request to the Alloy API endpoint to create the journey application:

```python
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer YourAccessTokenHere"
}

response = requests.post(url.format(journey_token=journey_token), json=payload, headers=headers)

if response.status_code == 201:
    print("Journey application created successfully!")
    print(response.json())
else:
    print("Failed to create journey application. Status code:", response.status_code)
    print(response.text)
```

Make sure to replace `"Bearer YourAccessTokenHere"` with the actual OAuth2 bearer token obtained through the authentication process.

This Python code snippet demonstrates how to create a journey application using the Alloy API.

### 👎 Guide for security authorization was not helpful

### 🤔 Does it produce significantl better results with gpt-4?

In [134]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

# for chunk in chunks:
#     print(len(yaml.safe_load(chunk.page_content)['paths']))

query = "How do I create a journey application in Python?"

chat = ChatOpenAI(model="gpt-4-turbo", temperature=0, model_kwargs={"seed": 0})

# Reset the collection to remove embeddings from previous runs
Chroma().delete_collection()

vectorstore = Chroma.from_documents(chunks, OpenAIEmbeddings())

In [135]:
metadata_field_info = [
    AttributeInfo(
        name="path",
        description="The subpath for this operation",
        type="string",
    ),
    AttributeInfo(
        name="method",
        description="The HTTP Method for this operation",
        type="string",
    ),
    AttributeInfo(
        name="tag",
        description="The logical grouping for this API operation",
        type="string",
    ),
    AttributeInfo(
        name="summary",
        description="A short description of this operation's functionality",
        type="string",
    ),
    AttributeInfo(
        name="description",
        description="A more detailed description of this operation's functionality",
        type="string",
    ),
]
document_content_description = "The pruned OpenAPI specification which includes only the relevant information for a particular operation in the OpenAPI specification."

# Dylan: really stupid, but I had to upgrade langchain to get this to work for some reason
retriever = SelfQueryRetriever.from_llm(
    chat, vectorstore, document_content_description, metadata_field_info, verbose=True, search_kwargs={"k": 2}
)

In [136]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

better_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a world-class solutions engineer and amazing at question-answering tasks. You are also a world-class API
            integrations specialist and deeply understand OpenAPI Specification.
            
            You strictly follow these rules:
            - If you don't know the answer, just say that you don't know.
            - Use markdown to ensure code blocks and commands are propertly formatted.
            - Make sure to always include environment setup instructions and code blocks that are helpful for a developer to copy-paste.
            - Always explain inputs and outputs of API requests. Be as detailed as possible.
            - When presented with an OpenAPI Specification and asked questions about it, always include necessary steps to
            authenticate with the endpoint.
            - When presented with multiple authentication options, make sure to guide the customer through both methods
            
            Answer the user's questions based on the below context:\n\n{context}""",
        ),
        (
            "user",
            "{question}"
        )
    ]
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | better_prompt
    | chat
    | StrOutputParser()
)

In [137]:
from IPython.display import display, Markdown
output = rag_chain.invoke(query)
display(Markdown(output))

To create a journey application using the Alloy API in Python, you can use either HTTP basic authentication or OAuth2 for authentication. Below, I'll guide you through both methods, including the necessary environment setup, Python code, and explanations of inputs and outputs.

### Prerequisites
1. Python installed on your system.
2. `requests` library installed. You can install it using pip if it's not already installed:
   ```bash
   pip install requests
   ```

### Using HTTP Basic Authentication

1. **Authentication Setup**:
   - You need your workflow token and secret to use HTTP Basic Authentication.

2. **Python Code**:
   ```python
   import requests
   from requests.auth import HTTPBasicAuth

   # API endpoint
   url = "https://demo-qasandbox.alloy.co/v1/journeys/{journey_token}/applications"

   # Replace `{journey_token}` with your actual journey token
   url = url.format(journey_token="YOUR_JOURNEY_TOKEN")

   # Your workflow token and secret
   workflow_token = "YOUR_WORKFLOW_TOKEN"
   workflow_secret = "YOUR_WORKFLOW_SECRET"

   # Request body
   payload = {
       "entities": [
           {
               "branch_name": "persons",
               "data": {
                   "addresses": [
                       {
                           "city": "New York",
                           "country_code": "US",
                           "line_1": "41 E. 11th",
                           "line_2": "2nd floor",
                           "postal_code": "10003",
                           "state": "NY",
                           "type": "primary"
                       }
                   ],
                   "birth_date": "1990-01-25",
                   "document_ssn": 111223333,
                   "email_address": "john@alloy.com",
                   "ip_address_v4": "42.206.213.70",
                   "meta": {
                       "user_type": "vip"
                   },
                   "name_first": "John",
                   "name_last": "Doe",
                   "name_middle": "Franklin",
                   "phone_number": 8443825569
               },
               "entity_type": "person",
               "external_entity_id": "my_system_entity_id_123"
           }
       ]
   }

   # Headers
   headers = {
       "Content-Type": "application/json"
   }

   # Send POST request
   response = requests.post(url, json=payload, auth=HTTPBasicAuth(workflow_token, workflow_secret), headers=headers)

   # Print response
   print(response.status_code)
   print(response.json())
   ```

### Using OAuth2

1. **Authentication Setup**:
   - Obtain a bearer token using the client credentials flow.

2. **Python Code to Get Bearer Token**:
   ```python
   token_url = "https://demo-qasandbox.alloy.co/v1/oauth/bearer"
   response = requests.post(token_url, auth=HTTPBasicAuth(workflow_token, workflow_secret))
   access_token = response.json().get('access_token')
   ```

3. **Python Code to Create Journey Application**:
   ```python
   headers = {
       "Authorization": f"Bearer {access_token}",
       "Content-Type": "application/json"
   }

   # Send POST request with the bearer token
   response = requests.post(url, json=payload, headers=headers)

   # Print response
   print(response.status_code)
   print(response.json())
   ```

### Inputs and Outputs
- **Inputs**: The request body should include details about the entities involved in the journey application. This includes personal information, addresses, and any other relevant data as structured in the payload.
- **Outputs**: The response will include details about the created journey application, such as the application token, status, and any links to further actions or details.

This setup should help you successfully create a journey application using the Alloy API in Python.

### 👍 Pretty good

Includes both options for authentication and copy-pastable code examples

### 🤔 What about making it so security credentials are pulled through environment variables

Lets try editing the system prompt

In [138]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

# for chunk in chunks:
#     print(len(yaml.safe_load(chunk.page_content)['paths']))

query = "How do I create a journey application in Python?"

chat = ChatOpenAI(model="gpt-4-turbo", temperature=0, model_kwargs={"seed": 0})

# Reset the collection to remove embeddings from previous runs
Chroma().delete_collection()

vectorstore = Chroma.from_documents(chunks, OpenAIEmbeddings())

In [139]:
metadata_field_info = [
    AttributeInfo(
        name="path",
        description="The subpath for this operation",
        type="string",
    ),
    AttributeInfo(
        name="method",
        description="The HTTP Method for this operation",
        type="string",
    ),
    AttributeInfo(
        name="tag",
        description="The logical grouping for this API operation",
        type="string",
    ),
    AttributeInfo(
        name="summary",
        description="A short description of this operation's functionality",
        type="string",
    ),
    AttributeInfo(
        name="description",
        description="A more detailed description of this operation's functionality",
        type="string",
    ),
]
document_content_description = "The pruned OpenAPI specification which includes only the relevant information for a particular operation in the OpenAPI specification."

# Dylan: really stupid, but I had to upgrade langchain to get this to work for some reason
retriever = SelfQueryRetriever.from_llm(
    chat, vectorstore, document_content_description, metadata_field_info, verbose=True, search_kwargs={"k": 2}
)

In [140]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

better_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a world-class solutions engineer and amazing at question-answering tasks. You are also a world-class API
            integrations specialist and deeply understand OpenAPI Specification. You also deeply unde
            rstand computer security which
            allows you to produce secure code examples and guidance.
            
            You strictly follow these rules:
            - If you don't know the answer, just say that you don't know.
            - Use markdown to ensure code blocks and commands are propertly formatted.
            - Make sure to always include environment setup instructions and code blocks that are helpful for a developer to copy-paste.
            - Always explain inputs and outputs of API requests. Be as detailed as possible.
            - When presented with an OpenAPI Specification and asked questions about it, always include necessary steps to
            authenticate with the endpoint.
            - When presented with multiple authentication options, make sure to guide the customer through both methods
            - Always name variables for security credentials based on standard protocols, API name, or descriptions
            - For example if the following security scheme is:
  "securitySchemes:
    basic:
      type: http
      description: HTTP basic authorization using a workflow token and secret
      scheme: Basic"

or

    "oauth2:
      type: oauth2
      x-default: viSRrSuUJEid8u0l3dyRTj5ATsWpHX9ShD51TH3j
      description: Oauth2 using a workflow token and secret to generate a bearer token
      flows:
        clientCredentials:
          tokenUrl: /oauth/bearer"
      
              Then you would name the variables in code as "ALLOY_WORKFLOW_TOKEN" and "ALLOY_WORKFLOW_SECRET".
            - When referencing security credentials, always pull values from environment variables.
            
            Answer the user's questions based on the below context:\n\n{context}""",
        ),
        (
            "user",
            "{question}"
        )
    ]
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | better_prompt
    | chat
    | StrOutputParser()
)

In [141]:
from IPython.display import display, Markdown
output = rag_chain.invoke(query)
display(Markdown(output))

To create a journey application using the Alloy API in Python, you'll need to authenticate and send a POST request to the appropriate endpoint. Below, I'll guide you through setting up your environment, authenticating using both Basic and OAuth2 methods, and making the API request.

### Environment Setup

First, ensure you have Python and `requests` library installed. You can install the `requests` library using pip if you haven't already:

```bash
pip install requests
```

### Authentication

The Alloy API provides two methods for authentication: Basic and OAuth2. I'll provide examples for both.

#### 1. Basic Authentication

For Basic Authentication, you need your workflow token and secret. These should be stored securely and not hard-coded in your scripts. Use environment variables to store these credentials.

```python
import os
import requests
from requests.auth import HTTPBasicAuth

# Environment variables for credentials
ALLOY_WORKFLOW_TOKEN = os.getenv('ALLOY_WORKFLOW_TOKEN')
ALLOY_WORKFLOW_SECRET = os.getenv('ALLOY_WORKFLOW_SECRET')

# API endpoint
url = "https://demo-qasandbox.alloy.co/v1/journeys/{journey_token}/applications"

# Replace {journey_token} with your actual journey token
url = url.format(journey_token="your_journey_token_here")

# Request body
payload = {
    "do_await_additional_entities": False,
    "entities": [
        {
            "branch_name": "persons",
            "data": {
                "addresses": [
                    {
                        "city": "New York",
                        "country_code": "US",
                        "line_1": "41 E. 11th",
                        "line_2": "2nd floor",
                        "postal_code": "10003",
                        "state": "NY",
                        "type": "primary"
                    }
                ],
                "birth_date": "1990-01-25",
                "document_ssn": 111223333,
                "email_address": "john@alloy.com",
                "ip_address_v4": "42.206.213.70",
                "meta": {
                    "user_type": "vip"
                },
                "name_first": "John",
                "name_last": "Doe",
                "name_middle": "Franklin",
                "phone_number": 8443825569
            },
            "entity_type": "person",
            "external_entity_id": "my_system_entity_id_123"
        }
    ]
}

# Headers
headers = {
    'Content-Type': 'application/json'
}

# Send POST request
response = requests.post(url, json=payload, auth=HTTPBasicAuth(ALLOY_WORKFLOW_TOKEN, ALLOY_WORKFLOW_SECRET), headers=headers)

# Print response
print(response.text)
```

#### 2. OAuth2 Authentication

For OAuth2, you'll first need to obtain a bearer token using the client credentials flow.

```python
# Get the bearer token
def get_bearer_token():
    token_url = "https://demo-qasandbox.alloy.co/v1/oauth/bearer"
    response = requests.post(token_url, auth=HTTPBasicAuth(ALLOY_WORKFLOW_TOKEN, ALLOY_WORKFLOW_SECRET))
    return response.json().get('access_token')

# Use the bearer token to authenticate your request
bearer_token = get_bearer_token()
headers = {
    'Authorization': f'Bearer {bearer_token}',
    'Content-Type': 'application/json'
}

# Send POST request
response = requests.post(url, json=payload, headers=headers)

# Print response
print(response.text)
```

### Explanation

- **URL**: Replace `"your_journey_token_here"` with the actual journey token you have.
- **Payload**: This JSON object contains the details about the entities involved in the journey application. Modify it according to your specific requirements.
- **Headers**: Necessary for indicating the content type and authorization method.

This setup should help you successfully create a journey application using the Alloy API in Python. Adjust the `payload` as per your data schema and requirements.

### 👍 Excellent

Looks like we should be using RAG (to reduce cost) + GPT-4 (for quality).

### 🤔 What about in Java?

In [146]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

# for chunk in chunks:
#     print(len(yaml.safe_load(chunk.page_content)['paths']))

query = "How do I create a journey application in Java?"

chat = ChatOpenAI(model="gpt-4-turbo", temperature=0, model_kwargs={"seed": 0})

# Reset the collection to remove embeddings from previous runs
Chroma().delete_collection()

vectorstore = Chroma.from_documents(chunks, OpenAIEmbeddings())

In [147]:
metadata_field_info = [
    AttributeInfo(
        name="path",
        description="The subpath for this operation",
        type="string",
    ),
    AttributeInfo(
        name="method",
        description="The HTTP Method for this operation",
        type="string",
    ),
    AttributeInfo(
        name="tag",
        description="The logical grouping for this API operation",
        type="string",
    ),
    AttributeInfo(
        name="summary",
        description="A short description of this operation's functionality",
        type="string",
    ),
    AttributeInfo(
        name="description",
        description="A more detailed description of this operation's functionality",
        type="string",
    ),
]
document_content_description = "The pruned OpenAPI specification which includes only the relevant information for a particular operation in the OpenAPI specification."

# Dylan: really stupid, but I had to upgrade langchain to get this to work for some reason
retriever = SelfQueryRetriever.from_llm(
    chat, vectorstore, document_content_description, metadata_field_info, verbose=True, search_kwargs={"k": 2}
)

In [148]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

better_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a world-class solutions engineer and amazing at question-answering tasks. You are also a world-class API
            integrations specialist and deeply understand OpenAPI Specification. You also deeply unde
            rstand computer security which
            allows you to produce secure code examples and guidance.
            
            You strictly follow these rules:
            - If you don't know the answer, just say that you don't know.
            - Use markdown to ensure code blocks and commands are propertly formatted.
            - Make sure to always include environment setup instructions and code blocks that are helpful for a developer to copy-paste.
            - Always explain inputs and outputs of API requests. Be as detailed as possible.
            - When presented with an OpenAPI Specification and asked questions about it, always include necessary steps to
            authenticate with the endpoint.
            - When presented with multiple authentication options, make sure to guide the customer through both methods of authentication in code
            - Always name variables for security credentials based on standard protocols, API name, or descriptions
            - For example if the following security scheme is:
  "securitySchemes:
    basic:
      type: http
      description: HTTP basic authorization using a workflow token and secret
      scheme: Basic"

or

    "oauth2:
      type: oauth2
      x-default: viSRrSuUJEid8u0l3dyRTj5ATsWpHX9ShD51TH3j
      description: Oauth2 using a workflow token and secret to generate a bearer token
      flows:
        clientCredentials:
          tokenUrl: /oauth/bearer"
      
              Then you would name the variables in code as "ALLOY_WORKFLOW_TOKEN" and "ALLOY_WORKFLOW_SECRET".
            - When referencing security credentials, always pull values from environment variables.
            
            Answer the user's questions based on the below context:\n\n{context}""",
        ),
        (
            "user",
            "{question}"
        )
    ]
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | better_prompt
    | chat
    | StrOutputParser()
)

In [149]:
from IPython.display import display, Markdown
output = rag_chain.invoke(query)
display(Markdown(output))

To create a journey application using the Alloy API in Java, you'll need to set up your environment and write code to make an HTTP POST request to the `/journeys/{journey_token}/applications` endpoint. Below, I'll guide you through the necessary steps, including handling authentication and making the API request.

### Environment Setup

1. **Java Development Kit (JDK):** Ensure you have Java installed. You can download it from [Oracle's website](https://www.oracle.com/java/technologies/javase-jdk11-downloads.html) or use OpenJDK.

2. **IDE (Integrated Development Environment):** You can use any IDE like IntelliJ IDEA, Eclipse, or VSCode for writing and executing your Java code.

3. **Dependency Management:** I recommend using Maven or Gradle to manage your dependencies. For this example, I'll use Maven.

4. **HTTP Client Library:** You will need an HTTP client library to make HTTP requests. For this example, I'll use Apache HttpClient, which you can include in your Maven `pom.xml`.

### Maven Setup

Add the following dependencies to your `pom.xml` file:

```xml
<dependencies>
    <dependency>
        <groupId>org.apache.httpcomponents</groupId>
        <artifactId>httpclient</artifactId>
        <version>4.5.13</version>
    </dependency>
    <dependency>
        <groupId>org.apache.httpcomponents</groupId>
        <artifactId>httpcore</artifactId>
        <version>4.4.14</version>
    </dependency>
    <dependency>
        <groupId>com.fasterxml.jackson.core</groupId>
        <artifactId>jackson-databind</artifactId>
        <version>2.12.3</version>
    </dependency>
</dependencies>
```

### Java Code Example

Here's how you can write Java code to create a journey application:

```java
import org.apache.http.HttpHeaders;
import org.apache.http.client.methods.CloseableHttpResponse;
import org.apache.http.client.methods.HttpPost;
import org.apache.http.entity.StringEntity;
import org.apache.http.impl.client.CloseableHttpClient;
import org.apache.http.impl.client.HttpClients;
import org.apache.http.util.EntityUtils;
import com.fasterxml.jackson.databind.ObjectMapper;

import java.util.Base64;

public class AlloyApiExample {
    private static final String BASE_URL = "https://demo-qasandbox.alloy.co/v1";
    private static final String JOURNEY_TOKEN = "your_journey_token_here";

    public static void main(String[] args) {
        try {
            createJourneyApplication();
        } catch (Exception e) {
            e.printStackTrace();
        }
    }

    private static void createJourneyApplication() throws Exception {
        String endpoint = String.format("%s/journeys/%s/applications", BASE_URL, JOURNEY_TOKEN);
        CloseableHttpClient client = HttpClients.createDefault();
        HttpPost httpPost = new HttpPost(endpoint);

        // Set headers
        String auth = System.getenv("ALLOY_WORKFLOW_TOKEN") + ":" + System.getenv("ALLOY_WORKFLOW_SECRET");
        byte[] encodedAuth = Base64.getEncoder().encode(auth.getBytes());
        String authHeader = "Basic " + new String(encodedAuth);
        httpPost.setHeader(HttpHeaders.AUTHORIZATION, authHeader);
        httpPost.setHeader(HttpHeaders.CONTENT_TYPE, "application/json");

        // Create request body
        String jsonBody = "{\"entities\":[{\"branch_name\":\"persons\",\"data\":{\"name_first\":\"John\",\"name_last\":\"Doe\"},\"entity_type\":\"person\"}]}";
        StringEntity entity = new StringEntity(jsonBody);
        httpPost.setEntity(entity);

        // Execute request
        try (CloseableHttpResponse response = client.execute(httpPost)) {
            String jsonResponse = EntityUtils.toString(response.getEntity());
            System.out.println("Response: " + jsonResponse);
        } finally {
            client.close();
        }
    }
}
```

### Explanation

1. **Authentication:** The code uses HTTP Basic Authentication. Replace `"your_journey_token_here"` with the actual journey token. Ensure that the environment variables `ALLOY_WORKFLOW_TOKEN` and `ALLOY_WORKFLOW_SECRET` are set with your credentials.

2. **HTTP POST Request:** The request is sent to the `/journeys/{journey_token}/applications` endpoint. Modify the JSON body as per your application requirements.

3. **Handling the Response:** The response from the server is printed out. You can modify this part to handle the response according to your business logic.

Make sure to replace placeholders and adjust parameters according to your specific use case and credentials.

In [150]:
query = "How do I create a journey application in TypeScript?"

In [151]:
from IPython.display import display, Markdown
output = rag_chain.invoke(query)
display(Markdown(output))

To create a journey application using the Alloy API in TypeScript, you'll need to follow these steps:

1. **Set up your environment**: Ensure you have Node.js and npm installed. You can download and install them from [Node.js official website](https://nodejs.org/).

2. **Create a new project**: Initialize a new Node.js project by running:
   ```bash
   mkdir alloy-journey-app
   cd alloy-journey-app
   npm init -y
   ```

3. **Install required packages**: Install Axios for making HTTP requests, and dotenv for managing environment variables.
   ```bash
   npm install axios dotenv
   ```

4. **Setup environment variables**: Create a `.env` file in your project root and add your credentials.
   ```plaintext
   ALLOY_WORKFLOW_TOKEN=your_workflow_token
   ALLOY_WORKFLOW_SECRET=your_workflow_secret
   ```

5. **Create the TypeScript file**: Create a file named `createJourneyApplication.ts`.

6. **Implement the API call**: Use the following TypeScript code to make a POST request to create a journey application.

```typescript
import axios from 'axios';
import * as dotenv from 'dotenv';

dotenv.config();

const alloyApiUrl = 'https://demo-qasandbox.alloy.co/v1';
const journeyToken = 'your_journey_token'; // Replace with your actual journey token

// Setup the data for the journey application
const data = {
  entities: [
    {
      branch_name: "persons",
      data: {
        addresses: [
          {
            city: "New York",
            country_code: "US",
            line_1: "41 E. 11th",
            line_2: "2nd floor",
            postal_code: "10003",
            state: "NY",
            type: "primary"
          }
        ],
        birth_date: "1990-01-25",
        document_ssn: 111223333,
        email_address: "john@alloy.com",
        ip_address_v4: "42.206.213.70",
        meta: {
          user_type: "vip"
        },
        name_first: "John",
        name_last: "Doe",
        name_middle: "Franklin",
        phone_number: 8443825569
      },
      entity_type: "person",
      external_entity_id: "my_system_entity_id_123"
    }
  ]
};

// Function to create a journey application
async function createJourneyApplication() {
  try {
    const response = await axios.post(`${alloyApiUrl}/journeys/${journeyToken}/applications`, data, {
      auth: {
        username: process.env.ALLOY_WORKFLOW_TOKEN!,
        password: process.env.ALLOY_WORKFLOW_SECRET!
      }
    });
    console.log('Journey Application Created:', response.data);
  } catch (error) {
    console.error('Error creating journey application:', error);
  }
}

createJourneyApplication();
```

7. **Compile and run your TypeScript code**: Ensure you have TypeScript installed globally or as a dev dependency, and then compile and run your TypeScript file.
   ```bash
   npm install -g typescript
   tsc createJourneyApplication.ts
   node createJourneyApplication.js
   ```

This script initializes a journey application using basic authentication. Make sure to replace `'your_journey_token'` with the actual journey token you intend to use. The data structure provided in the `data` variable should be adjusted according to the specific requirements and schema of your journey configuration in the Alloy API.

### 2️⃣ Add documentation and custom configuration to input

```mermaid
graph LR
    allow.com.yaml --> Context
    HWD[Hand-written Documentation] --> Context
    JSON[JSON Payload] --> Context
    Context --> Chat
    Query[How do I create a journey in Python?] --> Chat
    Chat --> Guide

```

In [6]:
import yaml
import jsonref
from jsonref import replace_refs
from langchain_core.documents.base import Document
from copy import deepcopy
from pprint import pprint

with open("alloy.com.yaml") as f:
    spec = f.read()

def chunk_openapi_by_operation(openapi: str):
    parsed = yaml.safe_load(openapi)

    operations: (str, str) = []
    # 1) list all operations by (path, HTTP method)
    for path, methods in parsed['paths'].items():
        for method in methods.keys():
            # if method is not an HTTP method then skip
            if method.lower() not in ['get', 'post', 'put', 'delete', 'patch', 'head', 'options', 'trace']:
                continue
            operations.append((path, method))

    # 2) create a chunk for every operation

    # 2.a) Dereference entire OpenAPI Spec
    dereferenced = replace_refs(parsed, lazy_load=False)

    chunks = []
    for operation in operations:
        path = operation[0]
        method = operation[1]
        chunk = deepcopy(dereferenced)
        if 'tags' in chunk['paths'][operation[0]][operation[1]]:
            tags = chunk['paths'][operation[0]][operation[1]]['tags']

        # first tag if possible
        if tags:
            tag_name = tags[0]

        # delete all tags on OAS except tag for this operation
        while len(chunk['tags']) > 1:
            for i in range(len(chunk['tags']) - 1, -1, -1):
                if chunk['tags'][i]['name'] != tag_name:
                    chunk['tags'].pop(i)

        if "summary" in chunk['paths'][path][method]:
            summary = chunk['paths'][path][method]['summary']
        else:
            summary = ""

        if "description" in chunk['paths'][path][method]:
            description = chunk['paths'][path][method]['description']
        else:
            description = ""

        # delete other operations
        for other_operation in operations:
            if other_operation[0] == operation[0]:
                continue
            if other_operation[0] in chunk['paths']:
                del chunk['paths'][other_operation[0]]

        # delete empty paths
        for path in chunk['paths'].keys():
            if not chunk['paths'][path]:
                del chunk['paths'][path]

        # delete other operations under same path
        keys = list(chunk['paths'][operation[0]].keys())
        for method in keys:
            if operation[1] == method:
                continue
            del chunk['paths'][operation[0]][method]

        # delete all components besides securitySchemes (should be inlined from 2.a)
        if "components" in chunk:
            for key in chunk["components"]:
                if key == "securitySchemes":
                    continue
                del chunk['components'][key]
        
        chunks.append(({
            "path": operation[0],
            "method": operation[1],
            "openapi": yaml.dump(chunk),
            "tag": tag_name,
            "summary": summary,
            "description": description
        }))
    return list(map(lambda chunk: Document(page_content=chunk["openapi"], metadata={
        "path": chunk["path"],
        "method": chunk["method"],
        "tag": chunk["tag"],
        "summary": chunk["summary"],
        "description": chunk["description"]
    }), chunks))
chunks = chunk_openapi_by_operation(spec)
# for chunk in chunks:
#     print(len(yaml.safe_load(chunk.page_content)['paths']))
print(len(chunks))

72


In [7]:
import phoenix as px

# Launch phoenix
session = px.launch_app()

# Once you have started a Phoenix server, you can start your LangChain application with the OpenInferenceTracer as a callback. To do this, you will have to instrument your LangChain application with the tracer:

from phoenix.trace.langchain import LangChainInstrumentor

# By default, the traces will be exported to the locally running Phoenix server.
LangChainInstrumentor().instrument()

session.url

Existing running Phoenix instance detected! Shutting it down and starting a new instance...
Attempting to instrument while already instrumented


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


'http://localhost:6006/'

In [8]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

# for chunk in chunks:
#     print(len(yaml.safe_load(chunk.page_content)['paths']))

query = "How do I create a journey application in Python?"

chat = ChatOpenAI(model="gpt-4-turbo", temperature=0, model_kwargs={"seed": 0})

# Reset the collection to remove embeddings from previous runs
Chroma().delete_collection()

vectorstore = Chroma.from_documents(chunks, OpenAIEmbeddings())

In [9]:
metadata_field_info = [
    AttributeInfo(
        name="path",
        description="The subpath for this operation",
        type="string",
    ),
    AttributeInfo(
        name="method",
        description="The HTTP Method for this operation",
        type="string",
    ),
    AttributeInfo(
        name="tag",
        description="The logical grouping for this API operation",
        type="string",
    ),
    AttributeInfo(
        name="summary",
        description="A short description of this operation's functionality",
        type="string",
    ),
    AttributeInfo(
        name="description",
        description="A more detailed description of this operation's functionality",
        type="string",
    ),
]
document_content_description = "The pruned OpenAPI specification which includes only the relevant information for a particular operation in the OpenAPI specification."

# Dylan: really stupid, but I had to upgrade langchain to get this to work for some reason
retriever = SelfQueryRetriever.from_llm(
    chat, vectorstore, document_content_description, metadata_field_info, verbose=True, search_kwargs={"k": 2}
)

In [10]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents.base import Document
from pprint import pprint
import json

better_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a world-class solutions engineer and amazing at question-answering tasks. You are also a world-class API
            integrations specialist and deeply understand OpenAPI Specification. You also deeply understand computer security which
            allows you to produce secure code examples and guidance. You are also a world-class technical documentation writer and make sure
            to include and details, edge cases, or options that the customer might consider.
            
            You strictly follow these rules:
            - If you don't know the answer, just say that you don't know.
            - Use markdown to ensure code blocks and commands are propertly formatted.
            - Make sure to always include environment setup instructions and code blocks that are helpful for a developer to copy-paste.
            - Always explain inputs and outputs of API requests. Be as detailed as possible.
            - When presented with an OpenAPI Specification and asked questions about it, always include necessary steps to
            authenticate with the endpoint.
            - When presented with multiple authentication options, make sure to guide the customer through both methods
            - Always name variables for security credentials based on standard protocols, API name, or descriptions
            - Make sure to include sections about required parameters 
            - If optional parameters are provided, explain them in detail should the customer need them.
            - For example if the following security scheme is:
  "securitySchemes:
    basic:
      type: http
      description: HTTP basic authorization using a workflow token and secret
      scheme: Basic"

or

    "oauth2:
      type: oauth2
      x-default: viSRrSuUJEid8u0l3dyRTj5ATsWpHX9ShD51TH3j
      description: Oauth2 using a workflow token and secret to generate a bearer token
      flows:
        clientCredentials:
          tokenUrl: /oauth/bearer"
      
              Then you would name the variables in code as "ALLOY_WORKFLOW_TOKEN" and "ALLOY_WORKFLOW_SECRET".
            - When referencing security credentials, always pull values from environment variables.
            
            Answer the user's questions based on the below context:\n\n

            OpenAPI Specification:
            {openapi}

            Hand-written documentation:
            {hwd}

            Configurations:
            {configurations}
            """,
        ),
        (
            "user",
            "{question}"
        )
    ]
)

with open("creating-a-journey.md") as f:
    hwd = f.read()
    
def format_docs(docs: list[Document]):
    return "\n\n".join(doc.page_content for doc in docs)

def format_hwd(docs: list[str]):
    return "\n\n".join(docs)

def format_configurations(configurations: list[dict]):
    return "\n\n".join(map(json.dumps, configurations))

def log_prompt(chat_prompt):
    for msg in chat_prompt:
        print(chat_prompt.to_string())
    return chat_prompt

rag_chain = (
    {
        "openapi": itemgetter("question") | retriever | format_docs,
        "question": itemgetter("question"),
        "hwd": itemgetter("documentation") | RunnableLambda(format_hwd),
        "configurations": itemgetter("configurations") | RunnableLambda(format_configurations)
    }
    | better_prompt
    | chat
    | StrOutputParser()
)

In [12]:
from IPython.display import display, Markdown

with open("creating-a-journey.md") as f:
    documentation = f.read()

configuration = {
  "journey_token": "J-VCQoADBJxeHtmdAvFqoS",
  "journey_version": "1",
  "timestamp": 1632400000,
  "data": [
    {
      "branch_name": "branch1",
      "workflows": [
        {
          "workflow_name": "workflow1",
          "parameters": {
            "required": [
              {
                "gender": True,
                "addresses": {
                  "state": True
                }
              }
            ],
            "optional": [
              {
                "document_license": True
              }
            ],
            "or": []
          }
        }
      ]
    }
  ]
}

output = rag_chain.invoke({"question": query, "documentation": [documentation], "configurations": [configuration]})
display(Markdown(output))

To create a Journey Application using the Alloy API in Python, you'll need to follow these steps. This involves setting up your environment, handling authentication, and making the API request with the necessary parameters.

### Environment Setup

First, ensure you have Python installed on your system. You can download Python from [python.org](https://www.python.org/downloads/). You will also need the `requests` library, which can be installed using pip:

```bash
pip install requests
```

### Authentication

The Alloy API provides two methods for authentication: Basic Authentication and OAuth2. Here's how you can handle both:

#### 1. Basic Authentication

For Basic Authentication, you will need your workflow token and secret. These should be stored securely and not hard-coded in your scripts.

```python
import requests
from requests.auth import HTTPBasicAuth
import os

# Environment variables for security credentials
ALLOY_WORKFLOW_TOKEN = os.getenv('ALLOY_WORKFLOW_TOKEN')
ALLOY_WORKFLOW_SECRET = os.getenv('ALLOY_WORKFLOW_SECRET')

# API endpoint
url = "https://demo-qasandbox.alloy.co/v1/journeys/{journey_token}/applications"

# Replace {journey_token} with your actual journey token
url = url.format(journey_token="J-VCQoADBJxeHtmdAvFqoS")

# Request headers
headers = {
    "Content-Type": "application/json"
}

# Request body
payload = {
    "entities": [
        {
            "branch_name": "persons",
            "data": {
                "name_first": "John",
                "name_last": "Doe",
                "birth_date": "1990-01-25",
                "document_ssn": 111223333,
                "email_address": "john@alloy.com",
                "phone_number": 8443825569,
                "addresses": [
                    {
                        "city": "New York",
                        "country_code": "US",
                        "line_1": "41 E. 11th",
                        "line_2": "2nd floor",
                        "postal_code": "10003",
                        "state": "NY",
                        "type": "primary"
                    }
                ],
                "gender": "male"
            },
            "entity_type": "person"
        }
    ]
}

# Make the request
response = requests.post(url, json=payload, headers=headers, auth=HTTPBasicAuth(ALLOY_WORKFLOW_TOKEN, ALLOY_WORKFLOW_SECRET))

# Print the response
print(response.json())
```

#### 2. OAuth2 Authentication

For OAuth2, you'll need to obtain a bearer token using your client credentials first.

```python
# Token endpoint
token_url = "https://demo-qasandbox.alloy.co/v1/oauth/bearer"

# Obtain the bearer token
token_response = requests.post(token_url, auth=HTTPBasicAuth(ALLOY_WORKFLOW_TOKEN, ALLOY_WORKFLOW_SECRET))
access_token = token_response.json().get('access_token')

# Update headers with the bearer token
headers['Authorization'] = f"Bearer {access_token}"

# Make the request with OAuth2 token
response = requests.post(url, json=payload, headers=headers)

# Print the response
print(response.json())
```

### Making the API Request

The example above shows how to make a request to create a Journey Application. You need to replace placeholders with actual data like `journey_token`, and ensure the payload matches the required schema as per your journey's configuration.

### Handling the Response

The response from the API will provide details about the created journey application, including tokens and status. Handle this response appropriately in your application to proceed with further actions or display the results to the user.

### 👎 This guide doesn't do a good job at explaining important details

### 🤔 Can we use [agentic workflows](https://www.youtube.com/watch?v=sal78ACtGTc&t=735s) to do better?